In [23]:
# ------------------------------------------
# Imports and setup
# ------------------------------------------

import sys, os
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

#Import the class EnvironmentalData
from data_import import EnvironmentalData
env_data=EnvironmentalData()
"""
Functions within the class, with arguments:
    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)
    get_nilu(self, threshold, file_path)
    missing_data(self, df)
    show_zeroes(self, df)
"""

'\nFunctions within the class, with arguments:\n    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)\n    get_nilu(self, threshold, file_path)\n    missing_data(self, df)\n    show_zeroes(self, df)\n'

In [24]:
# ------------------------------------------
# FORMATTING OF DATAFRAME VISUALISATION
# ------------------------------------------

# A function to make pandas show an entire dataframe in a readable manner
def pretty_data(df):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df)

In [25]:
# ------------------------------------------
# WEATHER DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the weather data
weather_station = 'SN68860' # The ID of the weather station to fetch data from, can be multiple stations separated by a comma - 'SN68860' is Voll weatherstation
weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)' # The measurements to include in the dataset, here: temperautre, precipitation, and wind speed
weather_time = '2024-01-30/2024-03-19' # The stretch of time to fetch weather data for
weather_resolution = 'P1D' # The level of granularity in the weather data. P1D is once per day. P1H is once per hour.

# Run the module for importing the weather data, and save it to a dataframe
df_weather = env_data.get_met(weather_station, weather_elements, weather_time, weather_resolution)

# Check if data was fetched successfully and check for missing values
if df_weather is None:
    print("Failed to fetch weather data.")
else:
    missing_data = env_data.missing_data(df_weather)
    print(missing_data)

# Displaying the weather data as a table
#pretty_data(df_weather)

Data collected from frost.met.no!
There are  245 lines of data in this dataframe.

No missing values found in the weather data! 

[]


In [30]:
# ------------------------------------------
# AIR QUALITY DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the air quality data
threshold = 95 # The threshold below which we will not use the data for analysis (uptime % of day)
file_path = '../Data/luftkvalitet_trondheim_dag.csv' #Choose the file to get air quality data from

# Run the module for importing the air quality data, and save it to a dataframe
df_quality = env_data.get_nilu(threshold, file_path)

# Check if data was fetched successfully and check for missing values
if df_quality is None:
    print("Failed to fetch weather data.")
else:
    missing_data = env_data.missing_data(df_quality)
    missing_df = pd.DataFrame(missing_data)  
    print(missing_df)

poor_quality_data = env_data.show_zeroes(df_quality)
print('These rows contain no data or data of too poor quality\n',poor_quality_data)

# Displaying the air quality data as a table
#pretty_data(df_quality)

Data collected from nilu.com!
There are  367 lines of data in this dataframe.

       0                          1
0    107     Elgeseter NO µg/m³ Day
1    107    Elgeseter NO2 µg/m³ Day
2    107    Elgeseter NOx µg/m³ Day
3    108     Elgeseter NO µg/m³ Day
4    108    Elgeseter NO2 µg/m³ Day
5    108    Elgeseter NOx µg/m³ Day
6    109     Elgeseter NO µg/m³ Day
7    109    Elgeseter NO2 µg/m³ Day
8    109    Elgeseter NOx µg/m³ Day
9    110     Elgeseter NO µg/m³ Day
10   110    Elgeseter NO2 µg/m³ Day
11   110    Elgeseter NOx µg/m³ Day
12   111     Elgeseter NO µg/m³ Day
13   111    Elgeseter NO2 µg/m³ Day
14   111    Elgeseter NOx µg/m³ Day
15   112     Elgeseter NO µg/m³ Day
16   112    Elgeseter NO2 µg/m³ Day
17   112    Elgeseter NOx µg/m³ Day
18   113     Elgeseter NO µg/m³ Day
19   113    Elgeseter NO2 µg/m³ Day
20   113    Elgeseter NOx µg/m³ Day
21   114     Elgeseter NO µg/m³ Day
22   114    Elgeseter NO2 µg/m³ Day
23   114    Elgeseter NOx µg/m³ Day
24   115     Elgesete